In [1]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import requests
import os
import psutil
from functions import *

2023-03-15 15:16:26.113996: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 15:16:26.255755: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-15 15:16:27.098616: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/bwhpc/common/devel/cuda/11.8/lib64
2023-03-15 15:16:27.098686: W tensorflow/compil

#### 1. Load BERT word embeddings 

In [10]:
files = os.listdir('nlm_embeddings/bert_wn18rr')
files = sorted(files)
files = files[1:]
files

['01_bert_4lastlayers_wn18rr_ent.pt',
 '02_bert_4lastlayers_wn18rr_ent.pt',
 '03_bert_4lastlayers_wn18rr_ent.pt',
 '04_bert_4lastlayers_wn18rr_ent.pt',
 '05_bert_4lastlayers_wn18rr_ent.pt',
 '06_bert_4lastlayers_wn18rr_ent.pt',
 '07_bert_4lastlayers_wn18rr_ent.pt',
 '08_bert_4lastlayers_wn18rr_ent.pt',
 '09_bert_4lastlayers_wn18rr_ent.pt',
 '10_bert_4lastlayers_wn18rr_ent.pt',
 '11_bert_4lastlayers_wn18rr_ent.pt',
 '12_bert_4lastlayers_wn18rr_ent.pt',
 '13_bert_4lastlayers_wn18rr_ent.pt',
 '14_bert_4lastlayers_wn18rr_ent.pt',
 '15_bert_4lastlayers_wn18rr_ent.pt',
 '16_bert_4lastlayers_wn18rr_ent.pt',
 '17_bert_4lastlayers_wn18rr_ent.pt']

In [13]:
embedd_raw = []
for file in files:
    batch_embedd = torch.load('nlm_embeddings/bert_wn18rr/' + file)
    batch_cpu = batch_embedd.to("cpu")
    embedd_raw.append(batch_cpu)
    del batch_embedd
    torch.cuda.empty_cache()

In [14]:
print("Virtual Memory percent: ", psutil.virtual_memory().percent)
print('GPU:  Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB |', 
          'Cached: ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Virtual Memory percent:  11.6
GPU:  Allocated: 17.8 GB | Cached:  17.8 GB


In [15]:
embeddings = []

for layer in range(len(embedd_raw[0])):
    lst = [embedd_raw[i][layer] for i in range(len(embedd_raw))]
    layer_embdd = torch.cat(lst, dim=0)
    embeddings.append(layer_embdd)

del embedd_raw

In [21]:
print("Virtual Memory percent: ", psutil.virtual_memory().percent)
print('GPU:  Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB |', 
          'Cached: ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Virtual Memory percent:  9.3
GPU:  Allocated: 8.9 GB | Cached:  8.9 GB


In [17]:
embedd_concat = concat_hidden_states(embeddings)
del embeddings
embedd_concat.size()

torch.Size([40943, 3072])